In [4]:
import torch
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from torch import nn
from torch.optim import Adam


In [9]:
import torch
data = pd.read_csv("train.csv")
train_size = int(len(data) * 0.8)
test_size = len(data) - train_size
train, test = data.iloc[0:train_size], data.iloc[train_size:len(data)]
Scaler = MinMaxScaler()
train_GHI = train["Clearsky GHI"].values
train.drop(["Clearsky GHI"],axis=1,inplace=True)
#train.drop(correlated_Columns,axis=1,inplace=True)
test_GHI = test["Clearsky GHI"].values

test.drop(["Clearsky GHI"],axis=1,inplace=True)
#test.drop(correlated_Columns,axis=1,inplace=True)


train["Datetime"] = pd.to_datetime(train[['Year', 'Month', 'Day', 'Hour', 'Minute']])
test["Datetime"] = pd.to_datetime(test[['Year', 'Month', 'Day', 'Hour', 'Minute']])

train.set_index("Datetime",inplace=True)
test.set_index("Datetime",inplace=True)

train.drop(["Year","Month","Day","Hour","Minute"],axis=1,inplace=True)
test.drop(["Year","Month","Day","Hour","Minute"],axis=1,inplace=True)

train = Scaler.fit_transform(train)
test = Scaler.fit_transform(test)

train_GHI = Scaler.fit_transform(train_GHI.reshape(-1,1))
test_GHI = Scaler.fit_transform(test_GHI.reshape(-1,1))

def createSequence(data,window_size):
    sequences = []
    for i in range(0,len(data) - window_size +1):
        sequence = data[i:i+window_size]
        sequences.append(sequence)
    return np.array(sequences)

sequences_96 = createSequence(train,window_size=96) # 2 day

train_GHI_96  = createSequence(train_GHI,window_size=96)
test_GHI_96 = createSequence(test_GHI,window_size=96)


sequences_96_tensor = torch.tensor(sequences_96).float().cuda()
train_GHI_96_tensor = torch.tensor(train_GHI_96).float().cuda()
test_GHI_96_tensor = torch.tensor(test_GHI_96).float().cuda()



C:\Users\burak\AppData\Local\Temp\ipykernel_16804\2484191830.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(["Clearsky GHI"],axis=1,inplace=True)
C:\Users\burak\AppData\Local\Temp\ipykernel_16804\2484191830.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(["Clearsky GHI"],axis=1,inplace=True)
C:\Users\burak\AppData\Local\Temp\ipykernel_16804\2484191830.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

In [10]:
class Model(nn.Module):
    def __init__(self, lr):
        super(Model, self).__init__()
        self.lstm = nn.LSTM(input_size=12, hidden_size=32, batch_first=True)
        self.fc1 = nn.Linear(32, 16)
        self.fc2 = nn.Linear(16, 1)
        self.relu = nn.ReLU()
        self.loss_function = nn.MSELoss()
        self.optimizer = Adam(self.parameters(), lr=lr)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return x

    def compile(self, x, y):
        self.optimizer.zero_grad()
        output = self.forward(x)
        loss = self.loss_function(output, y)
        loss.backward()
        self.optimizer.step()
        return loss.item()

In [15]:

def train_model(model, train_data, train_labels, test_data, test_labels, epochs, batch_size):
    train_loss = []
    test_loss = []
    for epoch in range(epochs):
        for i in range(0, len(train_data), batch_size):
            x = train_data[i:i + batch_size]
            y = train_labels[i:i + batch_size]
            loss = model.compile(x, y)
            train_loss.append(loss)
        with torch.no_grad():
            for i in range(0, len(test_data), batch_size):
                x = test_data[i:i + batch_size]
                y = test_labels[i:i + batch_size]
                pred = model.forward(x)
                loss = model.loss_function(pred, y)
                test_loss.append(loss.item())
        print("Epoch: {}, Train Loss: {}, Test Loss: {}".format(epoch+1, train_loss[-1], test_loss[-1]))
    return train_loss, test_loss

In [17]:
test_sequences_96_tensor = torch.tensor(createSequence(test,window_size=96)).float().cuda()
model = Model(lr=0.001).cuda()
train_loss, test_loss = train_model(model, sequences_96_tensor, train_GHI_96_tensor, test_sequences_96_tensor, test_GHI_96_tensor, 20, 16)

Epoch: 1, Train Loss: 1.5521791283390485e-05, Test Loss: 0.00025400909362360835
Epoch: 2, Train Loss: 6.147182375570992e-06, Test Loss: 0.00020034764020238072
Epoch: 3, Train Loss: 4.913292286801152e-06, Test Loss: 0.00019949831767007709
Epoch: 4, Train Loss: 3.333005679451162e-06, Test Loss: 0.00020945396681781858
Epoch: 5, Train Loss: 1.9532681108103134e-06, Test Loss: 0.00022133874881546944
Epoch: 6, Train Loss: 1.3612310567623354e-06, Test Loss: 0.00022364759934134781
Epoch: 7, Train Loss: 9.92446416603343e-07, Test Loss: 0.00023309812240768224
Epoch: 8, Train Loss: 7.295098498616426e-07, Test Loss: 0.0002447334991302341
Epoch: 9, Train Loss: 5.614995757241559e-07, Test Loss: 0.0002518611145205796
Epoch: 10, Train Loss: 4.2791435816980083e-07, Test Loss: 0.00025733516667969525
Epoch: 11, Train Loss: 3.7642951156158233e-07, Test Loss: 0.00026021338999271393
Epoch: 12, Train Loss: 5.095790811537881e-07, Test Loss: 0.0002741360804066062
Epoch: 13, Train Loss: 3.1628815122530796e-07, T

In [18]:
# Save model on  local
torch.save(model.state_dict(), "lstm.pth")